# 基于机器学习的刑事案件分类技术

## 文本预处理。

从语料库读取信息并进行文本清洗以及去停用词。

使用jieba以及nltk

In [16]:
#准备工作
import os
import win32com.client
import pandas as pd

## 转换格式

doc读取太慢，转为txt

In [17]:
import os
from win32com import client as wc
from docx import Document

def doc_to_docx(doc_file_path, docx_file_path):
    """
    使用 Word 应用程序将 .doc 文件转换为 .docx 文件
    :param doc_file_path: 输入的 .doc 文件路径
    :param docx_file_path: 输出的 .docx 文件路径
    """
    try:
        word = wc.Dispatch('Word.Application')
        doc = word.Documents.Open(doc_file_path)
        doc.SaveAs(docx_file_path, 12)  # 12 表示将 .doc 转为 .docx
        doc.Close()
        word.Quit()
        print(f"Successfully converted {doc_file_path} to {docx_file_path}")
    except Exception as e:
        print(f"Error converting file {doc_file_path}: {str(e)}")

def docx_to_txt(docx_file_path, txt_file_path):
    """
    使用 python-docx 将 .docx 文件转换为 .txt 文件
    :param docx_file_path: 输入的 .docx 文件路径
    :param txt_file_path: 输出的 .txt 文件路径
    """
    try:
        # 打开 .docx 文件
        doc = Document(docx_file_path)
        
        # 创建新的 .txt 文件
        with open(txt_file_path, 'w', encoding='utf-8') as txt_file:
            # 将 .docx 中的内容写入 .txt 文件
            for para in doc.paragraphs:
                txt_file.write(para.text)
                txt_file.write('\n')  # 添加换行符
        
        print(f"Successfully converted {docx_file_path} to {txt_file_path}")
    except Exception as e:
        print(f"Error converting file {docx_file_path}: {str(e)}")


def read_txt_file(txt_file_path):
    """
    读取 TXT 文件的内容。
    :param txt_file_path: 输入的 TXT 文件路径
    :return: 文件内容
    """
    with open(txt_file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text


def process_doc_files(doc_folder_path, txt_folder_path):
    """
    将文件夹中的所有 .doc 文件转换为 .txt 文件并读取其内容。
    :param doc_folder_path: 包含 .doc 文件的文件夹路径
    :param txt_folder_path: 用于保存 .txt 文件的文件夹路径
    :return: 包含所有文档内容的列表
    """
    if not os.path.exists(txt_folder_path):
        os.makedirs(txt_folder_path)
    
    all_texts = []
    
    for filename in os.listdir(doc_folder_path):
        if filename.endswith('.doc'):
            doc_file_path = os.path.join(doc_folder_path, filename)
            docx_file_path = os.path.join(doc_folder_path, filename.replace('.doc', '.docx'))
            txt_file_path = os.path.join(txt_folder_path, filename.replace('.doc', '.txt'))
            try:
                doc_to_docx(doc_file_path, docx_file_path)
                docx_to_txt(docx_file_path, txt_file_path)
                text = read_txt_file(txt_file_path)
                all_texts.append(text)
            except Exception as e:
                print(f"Error processing file {filename}: {e}")
    
    return all_texts


# 示例调用
doc_folder_path = 'D:/基于机器学习的刑事案件分类技术/test'
txt_folder_path = 'D:/基于机器学习的刑事案件分类技术/txt'
all_texts = process_doc_files(doc_folder_path, txt_folder_path)

Successfully converted D:/基于机器学习的刑事案件分类技术/test\（2011）兴刑初字第176号谢利杰非法出售珍贵、濒危野生动....doc to D:/基于机器学习的刑事案件分类技术/test\（2011）兴刑初字第176号谢利杰非法出售珍贵、濒危野生动....docx
Successfully converted D:/基于机器学习的刑事案件分类技术/test\（2011）兴刑初字第176号谢利杰非法出售珍贵、濒危野生动....docx to D:/基于机器学习的刑事案件分类技术/txt\（2011）兴刑初字第176号谢利杰非法出售珍贵、濒危野生动....txt
Successfully converted D:/基于机器学习的刑事案件分类技术/test\（2012）北刑一初字第31号刑事判决书.doc to D:/基于机器学习的刑事案件分类技术/test\（2012）北刑一初字第31号刑事判决书.docx
Successfully converted D:/基于机器学习的刑事案件分类技术/test\（2012）北刑一初字第31号刑事判决书.docx to D:/基于机器学习的刑事案件分类技术/txt\（2012）北刑一初字第31号刑事判决书.txt


### 读取txt和doc文件

In [18]:
def load_txt_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()


def load_files(directory):
    documents = []
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory, filename)
            documents.append(load_txt_file(file_path))
    return documents

## 清洗文本（使用正则表达式去除重复部分）

In [27]:
import re

# 正则表达式函数
def remove_header_footer(text):
    # 移除标题和页眉页脚等无关信息，可以根据具体情况定制正则表达式
    # 移除包含"人民法院"的子字符串
    text = re.sub(r'\b[\u4e00-\u9fa5]+人民法院\b', '', text)
    # 移除"刑 事 判 决 书"
    text = re.sub(r'刑\s*事\s*判\s*决\s*书', '', text)
    return text

def remove_case_number_and_date(text):
    # 移除案号和日期等信息
    text = re.sub(r'（\d+）[\u4e00-\u9fa5]{2,4}字第\d+号', '', text)
    text = re.sub(r'\d{4}年[\u4e00-\u9fa5]{2,4}月[\u4e00-\u9fa5]{2,4}日', '', text)
    return text

def remove_prosecution_office(text):
    # 移除公诉机关信息
    return re.sub(r'公诉机关[\u4e00-\u9fa5]+。', '', text)

def remove_judges_and_clerks(text):
    # 移除审判员和书记员信息
    text = re.sub(r'审判员[\u4e00-\u9fa5\s]+', '', text)
    return re.sub(r'书记员[\u4e00-\u9fa5\s]+', '', text)

def extract_labels(text):
    """
    从文本中提取罪名
    :param text: 输入文本
    :return: 罪名
    """
    # 正则表达式提取罪名
    # 定义匹配罪名的正则表达式
    crime_pattern = r'犯([\u4e00-\u9fa5]+罪)'
    
    # 使用正则表达式找到所有匹配项
    crime_matches = re.findall(crime_pattern, text)
    
    # 去除重复项并返回列表
    return list(set(crime_matches))

def remove_other_irrelevant_info(text):
# 移除包含"公安局"的子字符串
    text = re.sub(r'\b[\u4e00-\u9fa5]+公安局\b', '', text)
    # 移除包含"看守所"的子字符串
    text = re.sub(r'\b[\u4e00-\u9fa5]+看守所\b', '', text)
    return text

# 清洗函数
def clean_judgment_text(text):
    text = remove_header_footer(text)
    text = remove_case_number_and_date(text)
    text = remove_prosecution_office(text)
    text = remove_judges_and_clerks(text)
    label = extract_labels(text)
    text = remove_other_irrelevant_info(text)
    return text, label


### 读取停用词表。

In [20]:
# 加载中文停用词表
def load_stopwords(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        stop_words = set([line.strip() for line in file.readlines()])
    return stop_words

### 处理文本

由于实在是太多，采用了多线程进行处理

In [28]:
import jieba.posseg as pseg

def segment_and_filter(text, stopwords, allowed_pos):
    
    words_pos = pseg.cut(text)  # 分词并标注词性
    # 过滤停用词和不需要的词性
    filtered_words_pos = [word for word, flag in words_pos if word not in stopwords and flag in allowed_pos]

    return ' '.join(filtered_words_pos)


#### 运行函数

In [29]:
folder = 'D:/基于机器学习的刑事案件分类技术/txt'
stopwords = load_stopwords('D:/基于机器学习的刑事案件分类技术\\baidu_stopwords.txt')
allowed_pos = {'n', 'v', 'a', 'd'}
files = load_files(folder)
all_texts = []
labels = []
for file in files:
    text, label= clean_judgment_text(file)
    labels.append(label)
    all_texts.append(segment_and_filter(text, stopwords, allowed_pos))
print(all_texts)
print(labels)

['走私 毒品 犯', '因涉嫌 犯 珍贵 濒危 野生动物 罪 森林 逮捕 刑诉 字 起诉书 犯 珍贵 濒危 野生动物 本院 提起公诉 本院 依法 简易程序 实行 独任 开庭审理 到庭 参加 现已 终结 花鸟 市场 内 摆摊 黑 斑 头 均 属 国家 保护 动物 环颈雉 时许 公安人员 抓获 上述事实 开庭 过程 刑事案件 受理 抓获 现场 辨认 笔录 照片 扣押 物品 清单 动植物 物种 鉴定 证明 来源 野生动物 收条 户籍 证明 证实 本院认为 违反 国家 野生动物 保护 法规 国家 重点保护 珍贵 濒危 野生动物 已 珍贵 濒危 野生动物 归案 认罪态度 好 酌情 从轻 处罚 刑法 百 条 判决 犯 珍贵 濒危 野生动物 判处 拘役 判决 执行 计算 判决 执行 先行 羁押 羁押 折抵 起至 止 处罚金 人民币 罚金 判决 内 缴纳 不 缴纳 强制 缴纳 不服 判决 接到 判决书 内 本院 提出 书面 应 提交 上诉状 正本 副本', '原 上诉人 原审 男 日出 生于 小学文化 潜捕 船 所有人 船长 因涉嫌 犯 责任事故 逮捕 原审 责任事故 作出 原审 不服 提出 本院 依法 合议庭 审阅 案卷 材料 依法 讯问 原审 不 开庭审理 现已 终结 原判 认定 潜捕 船 所有人 船长 持有 驾驶员 职务 证书 驾驶 船舶 海域 返航 电建 渔港 进港 未 渔港 电建 大队 办理 签证 未 接受 不 履行 职责 未到 渔港 电建 大队 办理 签证 接受 情况 驾驶 船舶 搭载 黄 文 名 船员 均 办理 渔业 船 专业 基础训练 合格证书 不 出航 条件 情况 擅自 电建 渔港 出发 海域 潜捕 作业 时 分许 驾船 航行 东经 海域 船因 尾轴筒 受损 进水 连接 门 胶管 脱落 致使 海水 涌入 机舱 见状 文 报警 组织 全体人员 排水 自救 无效 潜捕 船 沉没 跳入 等待 获救 船员 溺水 死亡 船员 黄 失踪 公安 边防 总队 海警 支队 投案 家属 黄 达成 赔偿 协议 赔偿 被害人 约定 一次性 赔偿 失踪 已 支付 一次性 赔偿 失踪 已 支付 出狱 内 一次性 被害人 均 予以 谅解 原判 认定 上述事实 证人 证言 接受 刑事案件 户籍 证明 船 船员 持证 情况 渔业 船舶 所有权证 渔业 船舶 登记 证书 渔业 捕捞 渔业 船舶 证

### 将文本转化为TF-IDF向量。

In [ ]:
# 使用TfidfVectorizer生成TF-IDF特征向量
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(all_texts)

# 输出TF-IDF模型的特征名称和特征向量
print("TF-IDF模型特征向量:\n", X_tfidf.toarray())

### 使用PCA算法进行降维

经过TF-IDF算法处理后得到了一个多维的文本向量。

因为文本是多维度的，使用PCA算法对文本向量进行降维，保留最重要特征的同时简化向量，提高训练模型的性能

In [ ]:
# 使用PCA对TF-IDF特征向量进行降维
from sklearn.decomposition import PCA
n_components = 2 # 需要保留的主成分数量，可以根据实际情况调整
pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X_tfidf.toarray())

# 输出降维后的特征向量
print("降维后的特征向量:\n", X_pca)

# 输出保留的方差比例
print("各主成分的方差解释比例:", pca.explained_variance_ratio_)
print("总方差解释比例:", sum(pca.explained_variance_ratio_))

*进度:可以提取出罪名以及处罚作为标签，再划分测试集训练集。使用分类算法*